In [135]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from openai import OpenAI
from dotenv import load_dotenv
import os
import faiss

In [139]:
class RAG:
    def __init__(self):
        self.message=None
        self.chunk_mapping=None
        self.embedding_model=SentenceTransformer("models/all-MiniLM-L6-v2")
        self.index=None
        
    def generate_embeddings(self,sentence):
        embedding=self.embedding_model.encode(sentence)
        return embedding
    
    def embedding_of_data(self,file_name,chunk_size):
        with open(file=file_name,encoding="utf-8") as f:
            text=f.read()
        chunks=[]
        overlap=10
        step=chunk_size-overlap
        for i in range(0,len(text),step):
            chunks.append(text[i:i+chunk_size].strip())
            print(i/step)
        test_embedding=self.generate_embeddings("good day")
        dimension=test_embedding.shape[0]
        index=faiss.IndexFlatL2(dimension)
        for chunk in chunks:
            emb=self.generate_embeddings(chunk)
            index.add(np.array([emb]).astype("float32"))
        self.chunk_mapping=chunks
        self.index=index
        faiss.write_index(self.index,"index.faiss")
    
    def retrieve_top_k(self,query,k=5):
        query_emb=self.generate_embeddings(query)
        distances,indices=self.index.search(np.array([query_emb]).astype("float32"),k)
        return [self.chunk_mapping[i] for i in indices[0]]
    
    def llm_model(self,retrieved_context,query):
        retrieved_context_str = "\n".join(retrieved_context)
        load_dotenv()
        client = OpenAI(
            base_url="https://router.huggingface.co/v1",
            api_key=os.environ["HF_TOKEN"],
        )

        completion = client.chat.completions.create(
            model="Qwen/Qwen3-Next-80B-A3B-Instruct",
            messages = [
                {
                    "role": "system",
                    "content": (
                        "You are a helpful and concise AI assistant. "
                        "Answer questions clearly, accurately, and directly. "
                        "Do not mention 'based on the context' or similar phrases."
                    )
                },
                {
                    "role": "user",
                    "content": f"{retrieved_context_str}\n\nQuestion: {query}\nAnswer in a natural way without referring to the context explicitly."
                }
            ]
        )
        self.message=completion.choices[0].message.content

    def ask(self,prompt):
        context=self.retrieve_top_k(prompt)
        print(context)
        self.llm_model(context,prompt)
        print(self.message)

In [140]:
ragmodel=RAG()
ragmodel.embedding_of_data(file_name="data_iitj.txt",chunk_size=110)


0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0


In [141]:
ragmodel.ask("hi what is this data about?")

['ined on the Breast Cancer dataset\n\n🎯 Achieved 97% accuracy!\n\n🔍 What I learned:\n• Matrix shape debugging is pai', 'sequences.\n\n🔗 Features were concatenated and passed through dense layers to predict Kd.\n\n📊 Performance:\n\n✅ Tr', 'ew profile\n Message\n\n\n\n\n\n\n\nGirlScript Summer of Code\n\nIndian Institute of Technology Jodhpur\n\nPersonal Website', 'de\nJul 2025 - Present 3 months\n\nEducation\nIndian Institute of Technology Jodhpur Graphic\nIndian Institute of T', 'em mokal shared this\n🚀 Built a Neural Network from Scratch – 97% Accuracy on Breast Cancer Dataset!\n\nI challen']
This data is about a machine learning project where a neural network was built from scratch to classify breast cancer cases, achieving 97% accuracy. It involves analyzing medical features—likely from patient data like tumor size, cell texture, or other clinical measurements—to predict whether a case is malignant or benign. The model processed these features through dense layers, and the results 